In [1]:
from preprocessing import WaveformProcessor
import utilities
import numpy as np
import ROOT
from array import array
%jsroot on

Welcome to JupyROOT 6.30/04


# Defining Constants

In [2]:
WAVEFORM_BOUNDS = (1200, 1600)
INPUT_FILE = "/home/ryan/Documents/Research/Data/MilliQanWaveforms/outputWaveforms_812_2p5V.root"

# Plotting Data

In [3]:
processor = WaveformProcessor(INPUT_FILE)


# Set static bounds for LED datset
static_bounds = {key: np.array([[np.where(processor.times == WAVEFORM_BOUNDS[0])[0],
                                 np.where(processor.times == WAVEFORM_BOUNDS[1])[0]]])
                 for key in processor.histogram_dict}

# Isolate Waveforms
isolated_peaks = processor.isolate_waveforms(static_bounds)
peak_heights = np.max(isolated_peaks, axis=1)
hist = utilities.plot_histogram(peak_heights, 100, 0, 300,
                         write_to_existing_file=True,
                         file_path="height.root")
canvas = ROOT.TCanvas("canvas", "peak_height", 800, 600)
hist.Draw()
canvas.Draw()

Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1


# Fitting Function

In [4]:
par = array( 'd', 9*[0.] )
g1 = ROOT.TF1("g1", "gaus", 0, 100)
g2 = ROOT.TF1("g2", "gaus", 0, 100)
total = ROOT.TF1("total", "gaus(0)+gaus(3)", 0, 100)

total_fit = ROOT.TF1("total", "gaus(0)+gaus(3)", 0, 100)
total_fit.SetLineColor(4)
hist.Fit(g1, "NMS")
hist.Fit(g2, "NMS+")


 FCN=938.87 FROM HESSE     STATUS=OK             16 CALLS         259 TOTAL
                     EDM=5.72127e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     1.75947e+02   1.01758e+01   1.33292e-02  -8.68652e-05
   2  Mean         3.30174e+01   7.14950e-01   8.84518e-04   2.16803e-03
   3  Sigma        1.09816e+01   6.04434e-01   1.50200e-05  -9.49085e-02
 FCN=938.87 FROM HESSE     STATUS=OK             16 CALLS         259 TOTAL
                     EDM=5.72127e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     1.75947e+02   1.01758e+01   1.33292e-02  -8.68652e-05
   2  Mean         3.30174e+01   7.14950e-01   8.84518e-04   2.16803e-03
   3  Sigma        1.09816e+01   6.

In [5]:
par1 = g1.GetParameters()

par2 = g2.GetParameters()

par[0], par[1], par[2] = par1[0], 21, par1[2]
par[3], par[4], par[5] = par2[0], 39, par2[2]

total.SetParameters(par)
hist.Fit(total, 'RMS+')


 FCN=236.956 FROM HESSE     STATUS=OK             40 CALLS         523 TOTAL
                     EDM=1.87061e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           3.57521e+02   1.35085e+01   1.66723e-02   2.92854e-06
   2  p1           2.37507e+01   2.18525e-01   2.16201e-04  -5.42110e-04
   3  p2           4.14376e+00   1.33551e-01   1.08463e-04   8.29987e-04
   4  p3           9.97135e+01   5.44400e+00   5.19394e-03  -2.88327e-06
   5  p4           4.14919e+01   7.34051e-01   1.53426e-04  -3.26497e-07
   6  p5           1.24549e+01   5.25255e-01   5.58329e-04   3.88150e-04


In [6]:
hist.Draw()
canvas.Draw()

# Results
Looking at the above results the first NPE height should be around 24 and the second height should then be around 48